In [0]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import random
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

# 數據產生器，亂數產生出"100+150 "及 "250 "等數據

In [0]:
def creatData(length):
  que = []
  ans = []
  for i in range(length):
    x = random.randint(1,1000)
    y = random.randint(1,999)
    a = x+y
    str1 = str(x)+'+'+str(y)
    str2 = str(a)
    while len(str1)<8: 
      str1 = str1+' '
    while len(str2)<4: 
      str2 = str2+' '
    que.append(str1)
    ans.append(str2)
  return que,ans

In [0]:
dict1 = { ' ': [1,0,0,0,0,0,0,0,0,0,0,0],
         '+': [0,1,0,0,0,0,0,0,0,0,0,0],
         '0': [0,0,1,0,0,0,0,0,0,0,0,0],
         '1': [0,0,0,1,0,0,0,0,0,0,0,0],
         '2': [0,0,0,0,1,0,0,0,0,0,0,0],
         '3': [0,0,0,0,0,1,0,0,0,0,0,0],
         '4': [0,0,0,0,0,0,1,0,0,0,0,0],
         '5': [0,0,0,0,0,0,0,1,0,0,0,0],
         '6': [0,0,0,0,0,0,0,0,1,0,0,0],
         '7': [0,0,0,0,0,0,0,0,0,1,0,0],
         '8': [0,0,0,0,0,0,0,0,0,0,1,0],
         '9': [0,0,0,0,0,0,0,0,0,0,0,1]}



# 將文字依照上方字典做 Feature Engineering

In [0]:
def ceratDataSet(que,ans):
  #global = dict
  Q,A = [],[]
  for i in range(len(que)):
    a,b = [],[]
    for n in range(len(que[0])):
      a1 = dict1[que[i][n]]
      a = a+a1
    for k in range(len(ans[0])):
      a2 = dict1[ans[i][k]]
      b = b+a2
    Q.append(a)
    A.append(b)
  return np.array(Q),np.array(A)

# pytorch數據集

In [0]:
class MyDataset(Dataset):
  def __init__(self, qua, ans):
    self.Q = qua
    self.A = ans

  def __len__(self):
    return len(self.A)

  def __getitem__(self, idx):
      q = self.Q[idx]
      a = self.A[idx]
      return q,a

In [0]:
que,ans = creatData(100000)

persentage = 0.8

Q,A = ceratDataSet(que,ans)
fron = int(len(Q)*persentage)


Q_train = Q[:fron]
A_train = A[:fron]

Q_test = Q[fron:]
A_test = A[fron:]

train_dataloader = DataLoader(MyDataset(Q_train,A_train),batch_size=100, shuffle=True)
test_dataloader = DataLoader(MyDataset(Q_test,A_test), shuffle=True)

In [110]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


# 定義NN，這邊用6層 linear connection 做連接

In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.out1 = torch.nn.Linear(12*8,120)
    self.out2 = torch.nn.Linear(120,180)
    self.out3 = torch.nn.Linear(180,240)
    self.out4 = torch.nn.Linear(240,300)
    self.out5 = torch.nn.Linear(300,120)
    self.out6 = torch.nn.Linear(120,12*4)


  def forward(self, x):
    #print(self.out1.weight.type())
    #print(x.shape)
    x = self.out1(x)
    x = F.relu(x)
    x = self.out2(x)
    x = F.relu(x)
    x = self.out3(x)
    x = F.relu(x)
    x = self.out4(x)
    x = F.relu(x)
    x = self.out5(x)
    x = F.relu(x)
    x = self.out6(x)

    return x

In [112]:
model = CNN()

print(type(model))

if use_gpu:
  model = model.cuda()
print(model)

<class '__main__.CNN'>
CNN(
  (out1): Linear(in_features=96, out_features=120, bias=True)
  (out2): Linear(in_features=120, out_features=180, bias=True)
  (out3): Linear(in_features=180, out_features=240, bias=True)
  (out4): Linear(in_features=240, out_features=300, bias=True)
  (out5): Linear(in_features=300, out_features=120, bias=True)
  (out6): Linear(in_features=120, out_features=48, bias=True)
)


In [0]:
EPOCH = 50

loss_func = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

# 訓練 50 EPOCH

In [0]:
def train(epoch):
  for step, (x, y) in enumerate(train_dataloader):
      data = x.type(torch.cuda.FloatTensor)
      target = y.type(torch.cuda.FloatTensor)

      output = model(data)               # cnn output

      loss = loss_func(output, target)   # cross entropy loss
      optimizer.zero_grad()           # clear gradients for this training step
      loss.backward()                 # backpropagation, compute gradients
      optimizer.step()                # apply gradients

      if step % 200 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, 
         step * len(data), len(train_dataloader.dataset),100. * step / len(train_dataloader), loss.data.item()))
  print("Finish")

In [115]:
for epoch in range(1,EPOCH):
    train(epoch)

Train Epoch: 1 [0/80000 (0%)]	Loss: 0.088777
Train Epoch: 1 [20000/80000 (25%)]	Loss: 0.053467
Train Epoch: 1 [40000/80000 (50%)]	Loss: 0.047470
Train Epoch: 1 [60000/80000 (75%)]	Loss: 0.044307
Finish
Train Epoch: 2 [0/80000 (0%)]	Loss: 0.041437
Train Epoch: 2 [20000/80000 (25%)]	Loss: 0.039959
Train Epoch: 2 [40000/80000 (50%)]	Loss: 0.040483
Train Epoch: 2 [60000/80000 (75%)]	Loss: 0.039719
Finish
Train Epoch: 3 [0/80000 (0%)]	Loss: 0.039989
Train Epoch: 3 [20000/80000 (25%)]	Loss: 0.038601
Train Epoch: 3 [40000/80000 (50%)]	Loss: 0.038922
Train Epoch: 3 [60000/80000 (75%)]	Loss: 0.038507
Finish
Train Epoch: 4 [0/80000 (0%)]	Loss: 0.038948
Train Epoch: 4 [20000/80000 (25%)]	Loss: 0.038576
Train Epoch: 4 [40000/80000 (50%)]	Loss: 0.038885
Train Epoch: 4 [60000/80000 (75%)]	Loss: 0.037446
Finish
Train Epoch: 5 [0/80000 (0%)]	Loss: 0.036462
Train Epoch: 5 [20000/80000 (25%)]	Loss: 0.034798
Train Epoch: 5 [40000/80000 (50%)]	Loss: 0.034189
Train Epoch: 5 [60000/80000 (75%)]	Loss: 0.0340

# 逆Feature Engineering

In [0]:
def backtoNum(data,preprocess = True):
  p = ''
  if preprocess:
    data = np.array(data)
    if len(data)==48:
      data = data.reshape(4,12)
    elif len(data)==96:
      data = data.reshape(8,12)
    data = data.tolist()
    
  for i in range(len(data)):
    k = data[i].index(max(data[i]))
    if k == 0:
      a = ' '
    elif k == 1:
      a = '+'
    else:
      a = format(k-2)
    p = p+a
  return(p)

# 測試結果

In [0]:
def test():
  model.eval()
  correct = False
  c = 0
  for step, (x, y) in enumerate(test_dataloader):
    data = x.type(torch.cuda.FloatTensor)
    target = y.type(torch.cuda.FloatTensor)

    output = model(data)
    #print(output)


    da = backtoNum(data[0].cpu().detach().numpy())
    an = backtoNum(output[0].cpu().detach().numpy())
    co = backtoNum(target[0].cpu().detach().numpy())

    if an==co:
      correct = True
      c = c +1
    else:
      correct = False

    if step<20:
      print(da+" ?= "+an+' '+format(correct)+' '+co)
      
  p = c*100/len(test_dataloader)
  print("Accuracy : "+format(c)+'/'+format(len(test_dataloader))+' = '+format(p)+'%')
  print("Finish")

In [118]:
test()

520+921  ?= 1441 True 1441
322+869  ?= 1191 True 1191
770+681  ?= 1451 True 1451
467+527  ?= 993  False 994 
755+556  ?= 1311 True 1311
478+442  ?= 920  True 920 
536+567  ?= 1103 True 1103
591+322  ?= 913  True 913 
31+978   ?= 1009 True 1009
922+823  ?= 1745 True 1745
919+509  ?= 1428 True 1428
625+625  ?= 1250 True 1250
916+407  ?= 1323 True 1323
151+960  ?= 1111 True 1111
215+789  ?= 1004 True 1004
886+330  ?= 1216 True 1216
14+515   ?= 529  True 529 
897+290  ?= 1187 True 1187
782+361  ?= 1143 True 1143
686+362  ?= 1048 True 1048
Accuracy : 17264/20000 = 86.32%
Finish
